In [ ]:
include("_.jl")

In [ ]:
me_ge_ = OrderedDict{String,Vector{String}}()

;

In [ ]:
hg = Gene.read_hgnc()

pr_ = unique(skipmissing(hg[hg[!, "locus_group"].=="protein-coding gene", "symbol"]))

pr_ = sort(string.(pr_))

## Ensembl

In [ ]:
hu_mo = Gene.map_mouse(; di = "human_to_mouse")

me_ge_["Ensembl"] = [pr for pr in pr_ if !haskey(hu_mo, pr)]

## Mouse Genome Informatics

In [ ]:
mg = TableAccess.read(joinpath(PAI, "hom_mousehumansequence.rpt.txt.gz"))

se_ = mg[!, "Common Organism Name"] .== "human"

mg[se_, "Symbol"] .= Gene.rename(string.(mg[se_, "Symbol"]))[1]

hu_mo = Dict{String,String}()

for da in groupby(mg, "DB Class Key")

    sy_ = da[!, "Symbol"]

    se_ = da[!, "Common Organism Name"] .== "human"

    for hu in sy_[se_], mo in sy_[.!se_]

        hu_mo[hu] = mo

    end

end

println(length(hu_mo))

me_ge_["MGI"] = [pr for pr in pr_ if !haskey(hu_mo, pr)]

## 2011 De Novo Origin of Human Protein-Coding Genes

In [ ]:
tr_ = readlines(joinpath(PAI, "de_novo_origin_of_human_protein_coding_genes", "genes.txt"))

me_ge_["2011 paper"] = Gene.rename(vcat(["CLLU1", "c22orf45", "DNAH10OS"], tr_))[1]

## 2019 Genes with human-specific features are primarily involved with brain, immune and metabolic evolution

In [ ]:
an_ge_ir = TableAccess.read_xlsx(
    joinpath(
        PAI,
        "genes_with_human_specific_features_are_primarily_involved_with_brain,_immune_and_metabolic_evolution",
        "12859_2019_2886_MOESM2_ESM.xlsx",
    ),
    "HumanSpecific Genes",
)

me_ge_["2019 paper"] = Gene.rename(string.(an_ge_ir[!, "Gene Name"]))[1]

## Write

In [ ]:
write_gene_by_method(PAO, "human", me_ge_)